In [ ]:
print('init')

hi


In [ ]:
import anndata
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import scanpy as sc
from scipy.stats import median_abs_deviation
import seaborn as sns

In [ ]:
adatas = []
perSampleDir = Path('../data/raw/cellRangerOuts/BRI-2937/per_sample_outs/')
for experimentDir in perSampleDir.iterdir():
    adataPath = experimentDir / 'count' / 'sample_filtered_feature_bc_matrix.h5'
    adataSample = sc.read_10x_h5(adataPath)
    adataSample.var_names_make_unique()
    adataSample.obs['sample'] = experimentDir.stem
    adatas.append(adataSample)
adata = anndata.concat(adatas)

In [ ]:
# mitochondrial genes
adata.var["mt"] = adata.var_names.str.upper().str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.upper().str.startswith(("RPS", "RPL"))
# hemoglobin genes.
adata.var["hb"] = adata.var_names.str.upper().str.contains("^HB[^(P)]")

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True
)

In [ ]:
p1 = sns.displot(adata.obs["total_counts"], bins=100, kde=False)
# sc.pl.violin(adata, 'total_counts')
p2 = sc.pl.violin(adata, "pct_counts_mt")
p3 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

In [ ]:
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

In [ ]:
adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", 5)
    | is_outlier(adata, "log1p_n_genes_by_counts", 5)
    | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
)
adata.obs.outlier.value_counts()

In [ ]:
adata.obs["mt_outlier"] = is_outlier(adata, "pct_counts_mt", 3) | (
    adata.obs["pct_counts_mt"] > 8
)
adata.obs.mt_outlier.value_counts()

In [ ]:
print(f"Total number of cells: {adata.n_obs}")
adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()

print(f"Number of cells after filtering of low quality cells: {adata.n_obs}")

adata.layers["counts"] = adata.X


In [ ]:
import anndata2ri
import rpy2.robjects as ro
from rpy2.robjects.packages import importr

data_mat = adata.X.T
# samples = adata.obs["sample"]
importr("SingleCellExperiment")
importr("scDblFinder")
importr("BiocParallel")

detect_doublets = ro.functions.wrap_r_function(
    ro.r(
        """
function(data_mat){
print("data loaded, proceeding with doublet detection")
# bp <- MulticoreParam(cores, RNGseed=9)
sce = scDblFinder(
    SingleCellExperiment(
        list(counts=data_mat),
    )
)
return(
    list(
        score=sce$scDblFinder.score,
        class=sce$scDblFinder.class
    )
)
}
"""
    ),
    "detect_doublets",
)
with (
    ro.default_converter + ro.pandas2ri.converter + anndata2ri.converter
).context():
    outs = detect_doublets(data_mat)

adata.obs["scDblFinder_score"] = outs["score"]
adata.obs["scDblFinder_class"] = outs["class"]

In [ ]:
adata.write_h5ad('../data/interim/BRI-2937/BRI-2937_qc.h5ad')

In [ ]:
adata = sc.read_h5ad('../data/interim/BRI-2937/BRI-2937_qc.h5ad')

In [ ]:
# Shifted logarithm normalization
scales_counts = sc.pp.normalize_total(adata, target_sum=None, inplace=False)
adata.layers["log1p_norm"] = sc.pp.log1p(scales_counts["X"], copy=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
p1 = sns.histplot(adata.obs["total_counts"], bins=100, kde=False, ax=axes[0])
axes[0].set_title("Total counts")
p2 = sns.histplot(adata.layers["log1p_norm"].sum(1), bins=100, kde=False, ax=axes[1])
axes[1].set_title("Shifted logarithm")
plt.show()

In [ ]:
importr('scry')

featureSelection = ro.functions.wrap_r_function(
    ro.r(
        """
function(adata){
print("anndata loaded")
print(adata)
sce = devianceFeatureSelection(adata, assay="X")
return(rowData(sce)$binomial_deviance)
}
"""
    ),
    "featureSelection",
)
with (
    ro.default_converter + ro.pandas2ri.converter + anndata2ri.converter
).context():
    binomial_deviance = featureSelection(adata).T

In [ ]:
idx = binomial_deviance.argsort()[-4000:]
mask = np.zeros(adata.var_names.shape, dtype=bool)
mask[idx] = True

adata.var["highly_deviant"] = mask
adata.var["binomial_deviance"] = binomial_deviance

In [ ]:
sc.pp.highly_variable_genes(adata, layer="log1p_norm")

In [ ]:
ax = sns.scatterplot(
    data=adata.var, x="means", y="dispersions", hue="highly_deviant", s=5
)
ax.set_xlim(None, 1.5)
ax.set_ylim(None, 3)
plt.show()

In [ ]:
adata.write_h5ad('../data/interim/BRI-2937/BRI-2937_featureSelection.h5ad')

In [ ]:
adata = sc.read_h5ad('../data/interim/BRI-2937/BRI-2937_featureSelection.h5ad')

In [ ]:
adata.X = adata.layers["log1p_norm"]

In [ ]:
adata.var["highly_variable"] = adata.var["highly_deviant"]
sc.pp.pca(adata, svd_solver="arpack", use_highly_variable=True)
sc.pl.pca_scatter(adata, color="total_counts")

In [ ]:
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color="total_counts")

In [ ]:
sc.pl.umap(adata, color="sample")

In [ ]:
sc.pl.umap(
    adata,
    color=["total_counts", "pct_counts_mt", "scDblFinder_score", "scDblFinder_class"],
)

In [ ]:
adata.write_h5ad('../data/interim/BRI-2937/BRI-2937_reduced.h5ad')

In [ ]:
adata = sc.read_h5ad('../data/interim/BRI-2937/BRI-2937_reduced.h5ad')

In [ ]:
sc.pp.neighbors(adata, n_pcs=30)
sc.tl.umap(adata)

In [ ]:
sc.tl.leiden(adata, key_added="leiden_res0_25", resolution=0.25)
sc.tl.leiden(adata, key_added="leiden_res0_5", resolution=0.5)
sc.tl.leiden(adata, key_added="leiden_res1", resolution=1.0)

In [ ]:
sc.pl.umap(
    adata,
    color=["leiden_res0_25", "leiden_res0_5", "leiden_res1"],
    legend_loc="on data",
)

In [ ]:
# %%